In [13]:
from pathlib import Path
import pandas as pd

In [9]:
asc_file = Path('s04s07_AS_18Feb22.asc')
with asc_file.open() as file:
    samples = []
    events = {}
    is_recording_block = False
    for line in file:
        if line.isspace():
            continue
            
        line = line.split()
        if line[0] == 'START':
            is_recording_block = True
        if is_recording_block:
            if line[0].isdigit():
                samples.append(line)
            elif line[0].isupper():
                if line[0] not in events.keys():
                    events[f'{line[0]}'] = [line[1:]]
                else:
                    events[f'{line[0]}'].append(line[1:])
        if line[0] == 'END':
            is_recording_block = False

In [23]:
df_samples = pd.DataFrame(samples, columns=['time_stamp', 'x_pos', 'y_pos', 'pupil_size', 'input_pin', 'flags'])

In [20]:
df_blink = pd.DataFrame(events['EBLINK'], columns=['eye', 'start_blink', 'end_blink', 'duration'])